RitSpls Example
================

The `RitSpls` package is written to be consistent with `ScikitLearn.jl`, such that widely used functions from ScikitLearn can be applied to it (e.g., `SPLS` and `GSSPP` objects).


Install the `RitSpls` package and check the key functions:

In [ ]:
using RitSpls
print(isdefined(RitSpls, :SPLS))
print(isdefined(RitSpls, :GSSPP))
print(isdefined(RitSpls, :wrap))

typeof(predint)

1) Set up environment and load data
---------------------------------

Load dependencies

In [ ]:
using CSV
using DataFrames
using ScikitLearn
using ScikitLearnBase
import ScikitLearn.GridSearch:GridSearchCV

Load data

In [ ]:
# main_path = "YOUR_PATH"
# cd(main_path)
Xf = CSV.read("./data/Xfearncal.csv", DataFrame, header=0)
yf = CSV.read("./data/Yfearncal.csv", DataFrame, header=0)

# 2) Robustness-inducing transformations
--------------------------------------

Generalized spatial sign transformation of the predictors.

In [ ]:
# GSS pre-processing transformation
gsspp_X = RitSpls.GSSPP()
Xpp = ScikitLearn.fit_transform!(gsspp_X, Xf)

# compare with original (centered) predictors
loc = RitSpls.kstepLTS(Matrix(Xf))
Xcentered = autoscale(Matrix(Xf), loc, "none").X_as_
Xpp .- Xcentered

Wrapping transformation of the predictand:

In [ ]:
# wrapping pre-processing transformation
ywrap = RitSpls.wrap(yf)
ypp = ywrap.wrapX[1]

# compare with original predictand
ypp .- yf

3) SPLS on pre-processed data using cross-validation
-------------------

Estimate an SPLS model on robustly transformed data using cross validation.

In [ ]:
cv_folds = 2
n_comp_range = collect(1:4);
eta_range = collect(0.9:-0.2:0.1)

ritSPLSreg = RitSpls.SPLS()
RitSpls.set_params_dict!(ritSPLSreg, Dict(:fit_algorithm=>"snipls", :verbose => false))
gridsearch = GridSearchCV(ritSPLSreg, cv=cv_folds, Dict(:eta => 0.5, :n_components => 3))
solfit = ScikitLearn.fit!(gridsearch,Xpp,ypp)
fit!(solfit,Xpp,ypp)

println(solfit.best_estimator_.n_components)
println(solfit.best_estimator_.eta)